In [1]:
import numpy as np

from sklearn import preprocessing

# Load data
raw_csv_data = np.loadtxt('Audiobooks_data.csv',delimiter=',')

# Select inputs as all columns except the first and last column from dataset
unscaled_inputs_all = raw_csv_data[:,1:-1]

# Select targets as last column from dataset
targets_all = raw_csv_data[:,-1]

In [2]:
# Shuffle indices before balancing the dataset
shuffled_indices = np.arange(unscaled_inputs_all.shape[0])
np.random.shuffle(shuffled_indices)

# Use the shuffled indices to shuffle the unscaled inputs and targets.
shuffled_unscaled_inputs_all = unscaled_inputs_all[shuffled_indices]
shuffled_targets_all = targets_all[shuffled_indices]

In [3]:
# Count number of targets that are 1 
num_one_targets = int(np.sum(shuffled_targets_all))

# Set a counter for targets that are 0 
zero_targets_counter = 0

# Variable for removing some of the input/target pairs in order to balance dataset
indices_to_remove = []

# Function for counting the number of targets that are 0
# Once there are as many 0s as 1s, mark entries where the target is 0
for i in range(shuffled_targets_all.shape[0]):
    if shuffled_targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

# New variable to contain the inputs and the targets, while deleting all indices marked "to remove" in the loop above.
unscaled_balanced_inputs = np.delete(shuffled_unscaled_inputs_all, indices_to_remove, axis=0)
balanced_targets = np.delete(shuffled_targets_all, indices_to_remove, axis=0)

In [4]:
# Standardize inputs
scaled_inputs = preprocessing.scale(unscaled_balanced_inputs)

In [5]:
# Use shuffled indices to shuffle inputs and targets again before splitting data
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_scaled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = balanced_targets[shuffled_indices]

In [6]:
# Split dataset into train, validation, and test with 80-10-10 distribution
samples_count = shuffled_scaled_inputs.shape[0]

train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_scaled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_scaled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_scaled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

# Check if datasets are still balanced
print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1791.0 3579 0.5004191114836547
219.0 447 0.4899328859060403
227.0 448 0.5066964285714286


In [7]:
# Save the three datasets in *.npz
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)